In [196]:
import pandas
import plotnine

ModuleNotFoundError: No module named 'plotnine'

In [134]:
data = pandas.read_csv("10411_games_with_centipawn_metrics.csv")
data = data.drop(
    ["Unnamed: 0", "White Expected Rating by ACPL", "Black Expected Rating by ACPL"],
    axis=1,
)

In [83]:
table = {
    "Title": [
        "Super Grandmaster",
        "Grandmaster",
        "Master",
        "Expert Player",
        "Average Player",
        "Beginner",
    ],
    "ELO": ["2700+", "2500+", "2200-2499", "1800-2199", "1200-1799", "<1200"],
}

table = pandas.DataFrame.from_dict(table)


count = (
    pandas.cut(
        pandas.concat([data["White ELO"], data["Black ELO"]]),
        [0, 1200, 1800, 2200, 2500, 2700, 4000],
        labels=[
            "Beginner",
            "Average Player",
            "Expert Player",
            "Master",
            "Grandmaster",
            "Super Grandmaster",
        ],
    )
    .value_counts()
    .reset_index()
    .rename(columns={"index": "Title", 0: "Number of Games"})
)


pandas.merge(table, count)

,Title,ELO,Number of Games
0,Super Grandmaster,2700+,4661
1,Grandmaster,2500+,8303
2,Master,2200-2499,6348
3,Expert Player,1800-2199,1295
4,Average Player,1200-1799,184
5,Beginner,<1200,31


In [91]:
data.filter(like="*")
table = data["Result"].value_counts() / data["Result"].count()
print(table)

1/2-1/2    0.407820
1-0        0.353348
0-1        0.238832
Name: Result, dtype: float64

In [94]:
data[["White Label", "Black Label"]] = data[["White ELO", "Black ELO"]].apply(
    lambda x: pandas.cut(
        x,
        [0, 1200, 1800, 2200, 2500, 2700, 4000],
        labels=[
            "Beginner",
            "Average Player",
            "Expert Player",
            "Master",
            "Grandmaster",
            "Super Grandmaster",
        ],
    ),
    axis=0,
)
data[['White Label','Black Label','Result']]

,White Label,Black Label,Result
0,Master,Expert Player,1-0
1,Master,Grandmaster,1/2-1/2
2,Master,Master,0-1
3,Grandmaster,Master,1-0
4,Master,Master,0-1
...,...,...,...
10405,Grandmaster,Super Grandmaster,1/2-1/2
10406,Grandmaster,Grandmaster,1/2-1/2
10407,Super Grandmaster,Super Grandmaster,1/2-1/2
10409,Grandmaster,Super Grandmaster,1/2-1/2


In [147]:
from ast import literal_eval
data = pandas.read_csv("10411_games_with_centipawn_metrics.csv")

data['White CP Loss List'] = data['White CP Loss List'].apply(literal_eval)
data['Black CP Loss List'] = data['Black CP Loss List'].apply(literal_eval)

data['White CP Loss List'].explode()
data['Black CP Loss List'].explode()

0         4
0        53
0         7
0         5
0        42
         ..
10410    19
10410     0
10410    33
10410     0
10410    17
Name: White CP Loss List, Length: 460490, dtype: object

In [192]:
teste= pandas.DataFrame()
teste['Evaluations List'] = data['Evaluations List'].apply(literal_eval)

teste = teste.explode('Evaluations List').reset_index().rename(columns ={'index':'Game number'})
teste['Depth'] = teste.groupby('Game number').cumcount()




    Game number Evaluations List  Depth
0             0               47      0
1             0               43      1
2             0               44      2
3             0               -9      3
4             0               17      4
..          ...              ...    ...
73            0              994     73
74            0              994     74
75            0              995     75
76            0              995     76
77            0              996     77

[78 rows x 3 columns]


'c:\Users\Vitor' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
